In [ ]:
import gradio as gr
import pandas as pd
import random
from google.generativeai import GenerativeModel

# Load wine data
df = pd.read_csv("processed_wine_data.csv")

# Extract unique values for dropdown selections
color_options = df["Color"].dropna().unique().tolist()
aroma_options = df["aromas"].dropna().explode().unique().tolist()
flavor_options = df["Flavor"].dropna().explode().unique().tolist()
body_options = df["Body_Type"].dropna().unique().tolist()
alcohol_options = df["Alcohol_Content"].dropna().unique().tolist()
price_options = df["Price_Interval"].dropna().unique().tolist()

# Initialize Gemini AI
model = GenerativeModel("gemini-pro")

def recommend_wine(color, aroma, flavor, body, alcohol, price):
    """Filter wine dataset based on user selections and generate a recommendation."""
    filtered_df = df[
        (df["Color"] == color) &
        (df["aromas"].str.contains(aroma, na=False)) &
        (df["Flavor"].str.contains(flavor, na=False)) &
        (df["Body_Type"] == body) &
        (df["Alcohol_Content"] == alcohol) &
        (df["Price_Interval"] == price)
    ]
    
    if filtered_df.empty:
        return "No exact matches found. Try modifying your selections."
    
    selected_wine = filtered_df.sample(1).iloc[0]
    
    prompt = f"""
    Based on the following wine attributes:
    Color: {color}
    Aroma: {aroma}
    Flavor: {flavor}
    Body: {body}
    Alcohol Content: {alcohol}
    Price Range: {price}
    
    Recommend a wine from this dataset and provide a brief description:
    """
    
    ai_response = model.generate_content(prompt)
    recommendation = ai_response.text if ai_response else "No AI response received."
    
    return f"Wine Name: {selected_wine['Name']}\n\nDescription: {recommendation}\n\nMore info: {selected_wine['URL']}"

# Gradio interface
iface = gr.Interface(
    fn=recommend_wine,
    inputs=[
        gr.Dropdown(choices=color_options, label="Select Wine Color"),
        gr.Dropdown(choices=aroma_options, label="Select Aroma"),
        gr.Dropdown(choices=flavor_options, label="Select Flavor"),
        gr.Dropdown(choices=body_options, label="Select Body Type"),
        gr.Dropdown(choices=alcohol_options, label="Select Alcohol Content"),
        gr.Dropdown(choices=price_options, label="Select Price Range"),
    ],
    outputs=gr.Textbox(label="Recommended Wine"),
    title="Wine Recommendation App",
    description="Select your wine preferences and get a recommendation!",
)

iface.launch()
